In [ ]:
try:
  from transformer import pipeline
except Exception as e:
  print(e)
  print("installing...")
  !pip install datasets transformers[sentencepiece] -q
  from transformers import pipeline

No module named 'transformer'
installing...


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
%%timeit
classifier("I love my country")

58 ms ± 5.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def sentiment_classifier(in_text):
  return classifier(in_text)

demo = gr.Interface(fn=sentiment_classifier, inputs='text', outputs="text")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b7015696e101ce5a09.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
from transformers import AutoTokenizer

In [ ]:
ckpt = "distilbert-base-uncased-finetuned-sst-2-english"    # getting the weights of the model
tokenizer=AutoTokenizer.from_pretrained(ckpt)

In [ ]:
inputs = 'I love my country'

In [ ]:
tokenizer(inputs)

{'input_ids': [101, 1045, 2293, 2026, 2406, 102], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:
batch_input = ["I love my country",
               "I am Sai and I work in iNeuron"]

In [ ]:
out = tokenizer(batch_input, padding=True)
out

{'input_ids': [[101, 1045, 2293, 2026, 2406, 102, 0, 0, 0, 0, 0, 0], [101, 1045, 2572, 18952, 1998, 1045, 2147, 1999, 1999, 11236, 2239, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
type(out)

transformers.tokenization_utils_base.BatchEncoding

In [ ]:
# if you are developing applications using pytorch then go for pt tensors

out_pytorch_tensors = tokenizer(batch_input, padding=True, return_tensors='pt')    # pytorch
out_pytorch_tensors

{'input_ids': tensor([[  101,  1045,  2293,  2026,  2406,   102,     0,     0,     0,     0,
             0,     0],
        [  101,  1045,  2572, 18952,  1998,  1045,  2147,  1999,  1999, 11236,
          2239,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# if you are developing applications using tensorflow then go for tf tensors

out_TF_tensors = tokenizer(batch_input, padding=True, return_tensors='tf')    # tensorflow
out_TF_tensors

{'input_ids': <tf.Tensor: shape=(2, 12), dtype=int32, numpy=
array([[  101,  1045,  2293,  2026,  2406,   102,     0,     0,     0,
            0,     0,     0],
       [  101,  1045,  2572, 18952,  1998,  1045,  2147,  1999,  1999,
        11236,  2239,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 12), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
from transformers import AutoModel

In [ ]:
model_ckpt = AutoModel.from_pretrained(ckpt)

In [ ]:
model_ckpt

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [ ]:
logits = model_ckpt(**out_pytorch_tensors)
logits

BaseModelOutput(last_hidden_state=tensor([[[ 0.4521,  0.2584,  0.0448,  ...,  0.4836,  0.8697, -0.2664],
         [ 0.5710,  0.4134,  0.0491,  ...,  0.5090,  0.7748, -0.2068],
         [ 0.8872,  0.5492,  0.2826,  ...,  0.3853,  0.8563, -0.2052],
         ...,
         [ 0.1293,  0.1981, -0.0614,  ...,  0.5446,  0.7708, -0.1253],
         [ 0.0972,  0.2152, -0.0695,  ...,  0.5433,  0.7558, -0.1254],
         [ 0.2259,  0.4079, -0.0303,  ...,  0.4686,  0.7017, -0.1708]],

        [[ 0.5576,  0.2182,  0.2836,  ..., -0.2237,  0.6775, -0.1446],
         [ 0.9821,  0.7220,  0.3639,  ...,  0.0159,  0.7810, -0.3344],
         [ 0.4904,  0.9800,  0.8020,  ..., -0.0344,  0.5019, -0.1631],
         ...,
         [-0.7322,  0.2096,  0.4836,  ..., -0.4453,  0.5796, -0.5362],
         [-0.1331, -0.2090,  0.2910,  ..., -0.2575,  0.4169, -0.3687],
         [ 0.9476,  0.1276,  0.4248,  ...,  0.4503, -0.1079, -0.5512]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [ ]:
batch_size, sequence_length, hidden_state = logits['last_hidden_state'].shape
batch_size, sequence_length, hidden_state

(2, 12, 768)

In [ ]:
from transformers import AutoModelForSequenceClassification
model_2 = AutoModelForSequenceClassification.from_pretrained(ckpt)
pred = model_2(**out_pytorch_tensors)
pred

SequenceClassifierOutput(loss=None, logits=tensor([[-4.2268,  4.5594],
        [-2.5667,  2.6347]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
import torch
pred_prob = torch.nn.functional.softmax(pred.logits, dim=1)
print(pred_prob)

tensor([[1.5280e-04, 9.9985e-01],
        [5.4789e-03, 9.9452e-01]], grad_fn=<SoftmaxBackward0>)


In [ ]:

model_2.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [ ]:
torch.argmax(pred_prob, dim=-1)

tensor([1, 1])